<center>
<h1>Information Theory (Lab 1 excercise)</h1>
<h5>Maksymilian Norkiewicz</h5>
<h5>Poznan University of Technology</h5>
<h5>Lecturer: Iwo Błądek</h5>
<h5><b>October 5, 2024</b></h5>
<a href="https://github.com/Skamlo/Information-Theory">Full notes on github</a>
</center>

In [18]:
import numpy as np
import pandas as pd
import os

# 1. Preparation

In [75]:
file_names = os.listdir("./data")
files_text = []

for file_name in file_names:
    with open(f"./data/{file_name}", "r") as f:
        files_text.append(f.readline())

# 2. Zeroth-order approximation

In [8]:
class Zeroth_order_appx():
    def __init__(self, texts:list):
        self.signs = []
        self._list_unique_signs(texts)


    def _list_unique_signs(self, texts:list):
        for text in texts:
            self.signs = list(set(self.signs + list(text)))


    def _predict_sign(self):
        return np.random.choice(self.signs)
    

    def generate_message(self, n:int):
        return "".join([self._predict_sign() for i in range(n)])

In [11]:
zero_order = Zeroth_order_appx(files_text)
zero_order.generate_message(100)

'udbcjmbxpo6qc812s45 sotamhce6nhmg2djkenpkqhoi7x4errn0dxsv3y9797d6 o0gs26jhy7vd87ijulb2bmxdt5ch65ot3q'

### Average length of word

In [148]:
def calc_average_length(model, n:int):
    predicted_text = model.generate_message(n)
    words_lenght = [len(word) for word in predicted_text.split() if word != ""]
    average_lenght = sum(words_lenght) / len(words_lenght)
    return average_lenght

In [20]:
average_lenght = calc_average_length(zero_order, 10**6)
print(f"Average length of word in zeroth-order approximation is: ~{average_lenght:.0f}")

Average length of word in zeroth-order approximation is: ~37


# 3. Frequency of letters

In [12]:
frequency = {}
total_signs = 0
for text in files_text:
    for sign in list(text):
        if sign in frequency.keys():
            frequency[sign] += 1
        else:
            frequency[sign] = 1
        total_signs += 1

frequency = pd.DataFrame(frequency.values(), index=frequency.keys())
frequency.columns = ["Frequency"]
frequency["Frequency"] = frequency["Frequency"].transform(lambda x: x / total_signs)
frequency.sort_values("Frequency", inplace=True, ascending=False)
frequency["Frequency"] = frequency["Frequency"].transform(lambda x: f"{x*100:.2f}%")
frequency

,Frequency
,17.14%
e,9.35%
a,7.18%
t,6.65%
i,6.07%
n,5.94%
o,5.83%
r,5.41%
s,5.30%
h,3.69%


# 4. First-order approximation

In [76]:
class First_order_appx():
    def __init__(self, texts:list):
        self.signs_prob = {}
        self._calculate_prob(texts)


    def _calculate_prob(self, texts:list):
        # count signs
        total_signs = 0
        for text in texts:
            for sign in list(text):
                if sign in self.signs_prob.keys():
                    self.signs_prob[sign] += 1
                else:
                    self.signs_prob[sign] = 1
                total_signs += 1

        # count probabilities
        for sign in self.signs_prob.keys():
            self.signs_prob[sign] /= total_signs


    def _predict_sign(self):
        return np.random.choice(list(self.signs_prob.keys()), p=list(self.signs_prob.values()))
    

    def generate_message(self, n:int):
        return "".join([self._predict_sign() for i in range(n)])

In [77]:
first_order = First_order_appx(files_text)
first_order.generate_message(100)

'obr im fnn aohha o  eohgne1gepau fspt uo  iatts9sai   t0 2tso hrh latnhhdd nr8le hgo pl iwersm htisf'

### Average length of word

In [78]:
average_lenght = calc_average_length(first_order, 10**6)
print(f"Average length of word in first-order approximation is: ~{average_lenght:.0f}")

Average length of word in first-order approximation is: ~6


# 5. Conditional probability of letters

In [15]:
frequency = {}
for text in files_text:
    text = list(text)
    for i in range(1, len(text)):
        if text[i-1] in frequency.keys():
            if text[i] in frequency[text[i-1]].keys():
                frequency[text[i-1]][text[i]] += 1
            else:
                frequency[text[i-1]][text[i]] = 1
        else:
            frequency[text[i-1]] = {}

for sign_1 in frequency.keys():
    total_signs = sum(frequency[sign_1].values())
    for sign_2 in frequency[sign_1].keys():
        frequency[sign_1][sign_2] /= total_signs

frequency

{'t': {'r': 0.042739429190545436,
  ' ': 0.17729416061466358,
  'e': 0.12076716182438534,
  's': 0.02811007789196139,
  'f': 0.0008295622870937216,
  'h': 0.28404837257801074,
  'l': 0.009558295419214075,
  'a': 0.05400708454339453,
  'i': 0.10828028072061945,
  'o': 0.09251180870375122,
  'z': 0.0010467962738940414,
  'w': 0.008485702609387495,
  't': 0.01587708901026838,
  'c': 0.0038015947690055976,
  'u': 0.02239818175153048,
  'y': 0.02110699724248608,
  'n': 0.001545076731117275,
  'm': 0.002122104508555625,
  'b': 0.0021628358810806846,
  'p': 0.0005390118297482937,
  '5': 3.2585098020047984e-05,
  'd': 0.0003597937906380298,
  'v': 0.0012395914371793252,
  'j': 5.5666209117581967e-05,
  '1': 6.245477120509197e-05,
  'g': 0.0005580198035933216,
  'x': 4.208908494256197e-05,
  '2': 5.159307186507597e-05,
  'k': 0.0001805757515277659,
  '8': 3.1227385602545985e-05,
  '3': 3.2585098020047984e-05,
  '0': 4.073137252505998e-06,
  '4': 2.9869673185043982e-05,
  'q': 3.3942810437549984

# 6. Approximations based on Markov sources

In [140]:
class Markov_source_appx():
    def __init__(self, texts:list, order_level:int=1):
        self.signs_prob = {}
        self.order_level = order_level
        self._calculate_prob(texts, order_level)


    def _calculate_prob(self, texts:list, order_level:int):
        # count signs
        for text in texts:
            for i in range(len(text)-(order_level+1)):
                self._count_signs_chain(text[i:i+order_level+1], self.signs_prob)
        
        # count total n signs
        self._count_total_n_signs(order_level+1, self.signs_prob)
        self.signs_prob.pop("total")

        # count probabilities
        self._calculate_level_prob(order_level+1, self.signs_prob)


    def _count_signs_chain(self, signs:str, signs_prob:dict):
        if len(signs) == 1:
            if signs in signs_prob.keys():
                signs_prob[signs] += 1
            else:
                signs_prob[signs] = 1
        else:
            if not signs[0] in signs_prob.keys():
                signs_prob[signs[0]] = {}

            self._count_signs_chain(signs[1:], signs_prob[signs[0]])


    def _count_total_n_signs(self, order_level:int, signs_prob:dict):
        if order_level == 1:
            total = sum(signs_prob.values())
            signs_prob["total"] = total
        else:
            total = 0
            for sign in signs_prob.keys():
                self._count_total_n_signs(order_level-1, signs_prob[sign])
                total += signs_prob[sign]["total"]
            signs_prob["total"] = total
            

    def _calculate_level_prob(self, order_level:int, signs_prob:dict):
        if order_level == 1:
            n_elements = sum(signs_prob.values()) // 2 # division by 2 because signs_prob dictctionary have also "total" value
            signs = list(signs_prob.keys())
            signs.remove("total")
            for sign in signs:
                signs_prob[sign] /= n_elements
        else:
            n_elements = 0
            signs = list(signs_prob.keys())
            if "total" in signs:
                signs.remove("total")
            
            for sign in signs:
                n_elements += signs_prob[sign]["total"]
                self._calculate_level_prob(order_level-1, signs_prob[sign])

            for sign in signs:
                signs_prob[sign]["total"] /= n_elements


    def _predict_sign(self, previus_signs:str, order_level:int, signs_prob:dict):
        if order_level == 0:
            signs_prob_copy = signs_prob.copy()
            if "total" in signs_prob_copy.keys():
                signs_prob_copy.pop("total")
            return np.random.choice(list(signs_prob_copy.keys()), p=list(signs_prob_copy.values()))
        else:
            if len(previus_signs) == 0:
                signs = list(signs_prob.keys())
                if "total" in signs:
                    signs.remove("total")
                probs = [signs_prob[sign]["total"] for sign in signs]
                return np.random.choice(signs, p=probs)
            else:
                return self._predict_sign(previus_signs[1:], order_level-1, signs_prob[previus_signs[0]])


    def generate_message(self, n:int, starting_text:str=""):
        for i in range(n):
            starting_text += self._predict_sign(starting_text[-self.order_level:], self.order_level, self.signs_prob)
        return starting_text

In [143]:
markov_source_1_order = Markov_source_appx(files_text, 1)
markov_source_1_order_text = markov_source_1_order.generate_message(100)
markov_source_1_order_text

'aidarsce bakimicowiad h me g foren peincane me nd uluritallrepus ousleitofosherw ar fresthadetusstsp'

In [144]:
markov_source_3_order = Markov_source_appx(files_text, 3)
markov_source_3_order_text = markov_source_3_order.generate_message(100)
markov_source_3_order_text

'hia in york world would collujarand riginasa as exeted undeshot cal ash savaily who qasia cal with b'

In [145]:
markov_source_5_order = Markov_source_appx(files_text, 5)
markov_source_5_order_text = markov_source_5_order.generate_message(100, starting_text="probability")
markov_source_5_order_text

'probability heathers cup communicipality and becauses and the way hominoshima and s corps pixote answering vong'

### Average length of word

In [147]:
markov_source_1_order_average_lenght = calc_average_length(markov_source_1_order, 10**6)
markov_source_3_order_average_lenght = calc_average_length(markov_source_3_order, 10**6)
markov_source_5_order_average_lenght = calc_average_length(markov_source_5_order, 10**6)

print((
    "Average length of word in Markov sources approximation is:\n"
    "~{:.0f} for first-order\n"
    "~{:.0f} for third-order\n"
    "~{:.0f} for fifth-order"
).format(
    markov_source_1_order_average_lenght,
    markov_source_3_order_average_lenght,
    markov_source_5_order_average_lenght
))

Average length of word in Markov sources approximation is:
~5 for first-order
~5 for third-order
~5 for fifth-order


# Summary:

<table>
    <tr>
        <th>Model</th>
        <th>Average word lenght</th>
    </tr>
    <tr>
        <th>Zeroth-order approximation</th>
        <th>~37</th>
    </tr>
    <tr>
        <th>First-order approximation</th>
        <th>~6</th>
    </tr>
    <tr>
        <th>Markov source first-order</th>
        <th>~5</th>
    </tr>
    <tr>
        <th>Markov source third-order</th>
        <th>~5</th>
    </tr>
    <tr>
        <th>Markov source fifth-order</th>
        <th>~5</th>
    </tr>
</table>